# Cleaning and Preprocessing the Scopus publications related to COVID-19

For collecting the Scopus publications related to COVID-19, we used the "pybliometrics" library. It is avaliable on [https://pypi.org/project/pybliometrics/].

In [1]:
# Importing the required libraries.
import re, csv, pandas as pd, numpy as np
from pylatexenc.latex2text import LatexNodes2Text

## 1. Generating the dataframe from the raw data

In [2]:
# Creating a dataframe from the raw data.
df_data = pd.read_csv("../../data/raw/scopus_raw.csv", header=0,
    dtype={"id": "str", "eid": "str", "pii": "str", "pubmed_id": "str"})

In [3]:
# Checking the dataframe.
df_data.head()

,id,doi,eid,pii,pubmed_id,title,abstract,description,publication_date,citation_num,...,vehicle_name,vehicle_address,title_edition,publisher,affiliations,subject_areas,authors,author_affil,ref_count,references
0,85089569262,10.1109/LCSYS.2020.3009912,2-s2.0-85089569262,NaN,NaN,Can the COVID-19 Epidemic Be Controlled on the...,© 2017 IEEE.This letter studies if and to whic...,© 2017 IEEE.This letter studies if and to whic...,2021-07-01,0.0,...,IEEE Control Systems Letters,NaN,NaN,Institute of Electrical and Electronics Engine...,"({'id': '60023256', 'affiliation': 'Politecnic...","('Control and Systems Engineering', 'Control a...","({'id': '7004188711', 'name': 'Francesco Casel...","({'id': '7004188711', 'name': 'Francesco Casel...",27.0,"({'id': '85081266700', 'title': 'Characteristi..."
1,85089977035,10.1007/s11783-020-1328-8,2-s2.0-85089977035,NaN,NaN,Can we migrate COVID-19 spreading risk?,"© 2020, Higher Education Press.It is well reco...","© 2020, Higher Education Press.It is well reco...",2021-06-01,0.0,...,Frontiers of Environmental Science and Enginee...,NaN,NaN,Higher Education Press,"({'id': '60009254', 'affiliation': 'Purdue Uni...","('Environmental Science (all)',)","({'id': '57198483948', 'name': 'Qingyan Chen'},)","({'id': '57198483948', 'name': 'Qingyan Chen',...",10.0,"({'id': '85082406580', 'title': 'Turbulent gas..."
2,85090582917,10.1016/j.tourman.2020.104180,2-s2.0-85090582917,S0261517720301060,NaN,Beyond fragmentary: A proposed measure for tra...,© 2020The travel medicine literature points to...,© 2020The travel medicine literature points to...,2021-04-01,0.0,...,Tourism Management,NaN,NaN,Elsevier Ltd,"({'id': '60071895', 'affiliation': 'University...","('Development', 'Transportation', 'Tourism, Le...","({'id': '56411989300', 'name': 'Charles Atanga...","({'id': '56411989300', 'name': 'Charles Atanga...",118.0,"({'id': '84925321551', 'title': ""Backpackers' ..."
3,85090053297,10.1016/j.eswa.2020.113909,2-s2.0-85090053297,S0957417420307041,NaN,Coronavirus disease (COVID-19) detection in Ch...,© 2020 Elsevier LtdNovel coronavirus disease (...,© 2020 Elsevier LtdNovel coronavirus disease (...,2021-03-01,0.0,...,Expert Systems with Applications,NaN,NaN,Elsevier Ltd,"({'id': '60104578', 'affiliation': 'National I...","('Engineering (all)', 'Computer Science Applic...","({'id': '57169926900', 'name': 'Tej Bahadur Ch...","({'id': '57169926900', 'name': 'Tej Bahadur Ch...",66.0,"({'id': '85090055343', 'title': None, 'doi': N..."
4,85089501876,10.1016/j.jes.2020.07.029,2-s2.0-85089501876,S1001074220303405,NaN,Changes in air pollution during COVID-19 lockd...,© 2020The COVID-19 pandemic has escalated into...,© 2020The COVID-19 pandemic has escalated into...,2021-03-01,0.0,...,Journal of Environmental Sciences (China),NaN,NaN,Chinese Academy of Sciences,"({'id': '60105170', 'affiliation': 'Universida...","('Environmental Engineering', 'Environmental C...","({'id': '57210425621', 'name': 'Álvaro Briz-Re...","({'id': '57210425621', 'name': 'Álvaro Briz-Re...",67.0,"({'id': '85060059158', 'title': 'Impact of air..."


In [4]:
# Visualizing the information of dataset.
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53331 entries, 0 to 53330
Data columns (total 29 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                53331 non-null  object 
 1   doi               51796 non-null  object 
 2   eid               53328 non-null  object 
 3   pii               15890 non-null  object 
 4   pubmed_id         35058 non-null  object 
 5   title             53328 non-null  object 
 6   abstract          30202 non-null  object 
 7   description       30202 non-null  object 
 8   publication_date  53328 non-null  object 
 9   citation_num      53328 non-null  float64
 10  language          53044 non-null  object 
 11  production_type   53328 non-null  object 
 12  source_type       53328 non-null  object 
 13  auth_keywords     28021 non-null  object 
 14  index_terms       24011 non-null  object 
 15  issn              53238 non-null  object 
 16  isbn              142 non-null    object

## 2. Cleaning and preprocessing the dataframe

In [5]:
# Defining the function "clean_text" to clean and preprocess any text.
def clean_text(text, has_latex=False):
    if text:
        text = re.sub(r"\s+", " ", re.sub(r"\ufeff\.?", "", re.sub(
            r"\\\\(\’\s)?", "", str(text)))).replace("\u200b", "").replace("\ue001", "").replace(
            "\ue061", "").replace("\u202f", "").replace("\u2060", "").replace("\u200f", "").replace(
            "\u200e", "").replace("\u202c", "").replace("&#x2013;", "-").replace("&quot", "\"\"").replace(
            "\u200c", "").replace("\\u0019", "").replace("\\s", "s").strip()
        if has_latex:
            text = LatexNodes2Text().latex_to_text(re.sub("\\?%", "@PER@CENT@", text)).replace("@PER@CENT@", "%")
        text = re.sub(r"\s+", " ", re.sub(r"\-{2,}", "-", re.sub(r"\s?\xad(\s|\-)?", "-", text))).replace(
            "\\", "").replace("\\%", "%").replace("()", "").replace("[]", "").strip()
        return text
    else:
        return None

In [6]:
# Removing the invalid articles.
df_data = df_data.loc[df_data.id.notnull() & df_data.eid.notnull()]

In [7]:
# Defining the "None" value for the "NaN" values.
df_data.replace({np.nan: None}, inplace=True)

In [8]:
# Defining the "zero" value for the articles without numbers of citation and references.
df_data.citation_num.loc[df_data.citation_num.isnull()] = 0
df_data.ref_count.loc[df_data.ref_count.isnull()] = 0

In [9]:
# Normalizing the feature "abstract".
df_data.abstract.loc[df_data.abstract.isnull() & df_data.description.notnull()] = df_data.description.loc[
    df_data.abstract.isnull() & df_data.description.notnull()]
df_data.abstract.loc[df_data.abstract.notnull()] = df_data.abstract.loc[df_data.abstract.notnull()].apply(
    lambda x: clean_text(x, True))

In [10]:
# Normalizing the feature "vehicle_name".
df_data.vehicle_name.loc[df_data.conference_name.notnull() & df_data.vehicle_name.notnull()] = df_data.conference_name.loc[df_data.conference_name.notnull() & df_data.vehicle_name.notnull()]
df_data.vehicle_name.loc[df_data.vehicle_name.notnull()] = df_data.vehicle_name.loc[
    df_data.vehicle_name.notnull()].apply(clean_text)

In [11]:
# Normalizing the feature "title".
df_data.title.loc[df_data.title.notnull()] = df_data.title.loc[df_data.title.notnull()].apply(clean_text)

In [12]:
# Removing unnecessary columns.
columns_drop = ["eid", "pii", "description", "isbn", "conf_location", "conference_name",
    "vehicle_address", "title_edition"]
df_data.drop(axis=1, columns=columns_drop, inplace=True)

In [13]:
# Changing the type of some features.
df_data.loc[:, ["citation_num", "ref_count"]] = df_data.loc[
    :, ["citation_num", "ref_count"]].astype("int")
df_data.auth_keywords.loc[df_data.auth_keywords.notnull()] = df_data.auth_keywords.loc[
    df_data.auth_keywords.notnull()].apply(eval)
df_data.index_terms.loc[df_data.index_terms.notnull()] = df_data.index_terms.loc[
    df_data.index_terms.notnull()].apply(eval)
df_data.affiliations.loc[df_data.affiliations.notnull()] = df_data.affiliations.loc[
    df_data.affiliations.notnull()].apply(eval)
df_data.subject_areas.loc[df_data.subject_areas.notnull()] = df_data.subject_areas.loc[
    df_data.subject_areas.notnull()].apply(eval)
df_data.authors.loc[df_data.authors.notnull()] = df_data.authors.loc[df_data.authors.notnull()].apply(eval)
df_data.author_affil.loc[df_data.author_affil.notnull()] = df_data.author_affil.loc[
    df_data.author_affil.notnull()].apply(eval)
df_data.references.loc[df_data.references.notnull()] = df_data.references.loc[
    df_data.references.notnull()].apply(eval)
df_data.publication_date = pd.to_datetime(df_data.publication_date)

In [14]:
# Normalizing the itens contained in the features "auth_keywords" and "index_terms".
df_data.auth_keywords.loc[df_data.auth_keywords.notnull()] = df_data.auth_keywords.loc[
    df_data.auth_keywords.notnull()].apply(lambda x: tuple([clean_text(item) for item in x]))
df_data.index_terms.loc[df_data.index_terms.notnull()] = df_data.index_terms.loc[
    df_data.index_terms.notnull()].apply(lambda x: tuple([clean_text(item) for item in x]))

In [15]:
# Checking there are invalid values in the features "auth_keywords", "index_terms" and "subject_areas".
for column in ["auth_keywords", "index_terms", "subject_areas"]:
    count = df_data.loc[df_data[column].notnull(), column][
                [np.any([item == None or item.lower() == "none" for item in items])
                 for items in df_data.loc[df_data[column].notnull(), column]]].size
    print("{}: {}".format(column, count))

auth_keywords: 0
index_terms: 0
subject_areas: 0


In [16]:
# Removing the invalid values in the features "auth_keywords", "index_terms" and "subject_areas".
for column in ["auth_keywords", "index_terms", "subject_areas"]:
    df_data.loc[df_data[column].notnull(), column] = [
        tuple([item for item in items if item])
        for items in df_data.loc[df_data[column].notnull(), column]]
    df_data.loc[df_data[column].notnull(), column] = df_data.loc[
        df_data[column].notnull(), column].apply(lambda x: x if len(x) > 0 else None)

In [17]:
# Normalizing the content contained in the features "authors", "affiliations" and "author_affil".
df_data.affiliations.loc[df_data.affiliations.notnull()] = df_data.affiliations.loc[
    df_data.affiliations.notnull()].apply(lambda x: tuple([{"id": item["id"],
        "affiliation": clean_text(item["affiliation"]), "country": item["country"]}
        for item in x if item["id"]]))
df_data.author_affil.loc[df_data.author_affil.notnull()] = df_data.author_affil.loc[
    df_data.author_affil.notnull()].apply(lambda x: tuple(
        [{"id": item["id"], "name": clean_text(item["name"]), "affil_id": item["affil_id"],
          "affiliation": clean_text(item["affiliation"]), "country": item["country"]}
         for item in x if item["id"] or item["name"] or item["affil_id"] or \
             item["affiliation"] or item["country"]]))
df_data.authors.loc[df_data.authors.notnull()] = df_data.authors.loc[
    df_data.authors.notnull()].apply(lambda x: tuple(
        [{"id": item["id"], "name": clean_text(item["name"])} for item in x if item["id"]]))

In [18]:
# Creating the log file with invalid authors' Scopus ID (there is a Scopus ID, but there is not a name).
ids_authors = set([author["id"] for authors in df_data.authors.loc[df_data.authors.notnull()]
    for author in authors if author["id"] and not author["name"]])
ids_authors = ids_authors.union(set([author["id"] for authors in df_data.author_affil.loc[
        df_data.authors.isnull() & df_data.author_affil.notnull()]
    for author in authors if author["id"] and not author["name"]]))
with open("ids_invalids.txt", "w") as file:
    file.writelines(["{}\n".format(id_au) for id_au in ids_authors])

In [19]:
# Removing the authors without their complete name.
df_data.authors.loc[df_data.authors.notnull()] = df_data.authors.loc[
    df_data.authors.notnull()].apply(lambda x: tuple([item for item in x if item["name"]]))
df_data.author_affil.loc[df_data.author_affil.notnull()] = df_data.author_affil.loc[
    df_data.author_affil.notnull()].apply(lambda x: tuple([item for item in x
        if item["name"] or item["affil_id"] or item["affiliation"] or item["country"]]))

In [20]:
# Removing the invalid values in the features "authors", "affiliations" and "author_affil".
for column in ["authors", "affiliations", "author_affil"]:
    df_data.loc[df_data[column].notnull(), column] = df_data.loc[
        df_data[column].notnull(), column].apply(lambda x: x if len(x) > 0 else None)

In [21]:
# Creating the affiliations' and authors' IDs for those that have not a ID.
df_data.author_affil.loc[df_data.author_affil.notnull()] = df_data.author_affil.loc[
    df_data.author_affil.notnull()].apply(lambda x: tuple([{
        "id": item["id"] if item["name"] else None,
        "name": item["name"],
        "affil_id": item["affil_id"] if item["affil_id"] and item["affiliation"] else \
            str(hash("{} - {}".format(item["affiliation"], "Scopus"))) \
                if item["affiliation"] else None,
        "affiliation": item["affiliation"], "country": item["country"]}
    for item in x]))

In [22]:
# Removing the duplicated records by features "title" and "doi".
df_data = pd.concat([df_data[df_data.title.isnull() | df_data.doi.isnull()],
    df_data[df_data.title.notnull() & df_data.doi.notnull()].sort_values(
        by=["title", "citation_num", "publication_date"]).drop_duplicates(
            ["title", "doi"], "last")], ignore_index=True)

In [23]:
# Normalizing the feature "references".
df_data.references.loc[df_data.references.notnull()] = df_data.references.loc[
    df_data.references.notnull()].apply(lambda x: tuple(
        [{"id": ref["id"], "title": clean_text(ref["title"], True),
          "doi": clean_text(ref["doi"]), "authors": clean_text(ref["authors"], True)}
         for ref in x]))

In [24]:
# Checking the result.
df_data.head()

,id,doi,pubmed_id,title,abstract,publication_date,citation_num,language,production_type,source_type,...,index_terms,issn,vehicle_name,publisher,affiliations,subject_areas,authors,author_affil,ref_count,references
0,85090092938,None,None,12th International Conference on Intelligent N...,The proceedings contain 48 papers. The special...,2021-01-01,0,eng,Book Series,k,...,None,21945365 21945357,12th International Conference on Intelligent N...,Springer,None,"(Control and Systems Engineering, Computer Sci...",None,None,0,None
1,85090051630,None,None,23rd International Conference on Network-Based...,The proceedings contain 61 papers. The special...,2021-01-01,0,eng,Book Series,k,...,None,21945365 21945357,23rd International Conference on Network-Based...,Springer,None,"(Control and Systems Engineering, Computer Sci...",None,None,0,None
2,85090760353,None,None,Open educational resources in Canada 2020,"© 2020, Canadian Network for Innovation in Edu...",2020-12-01,0,eng,Journal,j,...,None,14996685,Canadian Journal of Learning and Technology,Canadian Network for Innovation in Education,"({'id': '60018510', 'affiliation': 'Athabasca ...","(Education, Computer Science Applications, Man...","({'id': '6507436970', 'name': 'Rory McGreal'},)","({'id': '6507436970', 'name': 'Rory McGreal', ...",25,"({'id': '85090794360', 'title': None, 'doi': N..."
3,85082677914,None,None,Industry reacts to coronavirus outbreak: Nonwo...,None,2020-12-01,0,eng,Trade Journal,d,...,None,01634429,Nonwovens Industry,Rodman Publications Inc.,None,"(Business and International Management, Materi...","({'id': '57193877330', 'name': 'Tara Olivo'},)","({'id': '57193877330', 'name': 'Tara Olivo', '...",0,None
4,85090891903,None,None,Proceedings of the 6th International Conferenc...,The proceedings contain 105 papers. The topics...,2020-09-14,0,eng,Conference Proceeding,p,...,None,None,6th International Conference on Engineering an...,Association for Computing Machinery,None,"(Computer Networks and Communications, Compute...",None,None,0,None


In [25]:
# Visualizing the information of dataset.
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53266 entries, 0 to 53265
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                53266 non-null  object        
 1   doi               51734 non-null  object        
 2   pubmed_id         35006 non-null  object        
 3   title             53266 non-null  object        
 4   abstract          30189 non-null  object        
 5   publication_date  53266 non-null  datetime64[ns]
 6   citation_num      53266 non-null  object        
 7   language          52982 non-null  object        
 8   production_type   53266 non-null  object        
 9   source_type       53266 non-null  object        
 10  auth_keywords     28011 non-null  object        
 11  index_terms       23975 non-null  object        
 12  issn              53176 non-null  object        
 13  vehicle_name      53266 non-null  object        
 14  publisher         5326

## 3. Saving the dataframe

In [26]:
# Exporting the data to CSV file.
df_data.to_csv("../../data/prepared/scopus_covid_19.csv", index=False, quoting=csv.QUOTE_ALL)